In [ ]:
import numpy as np
import os
import pylab
import seaborn
import scipy.stats as sp
import pickle
import imageio
from scipy.spatial import distance
import json
from collections import OrderedDict
import os.path as osp
import pandas as pd
import copy
import scipy.stats as ss
import sys
import matplotlib.pyplot as plt
DIV_LINE_WIDTH = 50
print(np.__version__)
print(sys.executable)

In [ ]:
##### BUILDING CLASSROOM STEP 0 - LOAD ALPGMM DATA
##### Make sure your data is in the data/ folder
exp_idx = 0
units = dict()

def get_all_runs(logdir, condition=None, minlen=20):
    #print(logdir)
    """
    Recursively look through logdir for output files produced by
    Assumes that any file "progress.txt" is a valid hit. 
    """
    global exp_idx
    global units
    datasets = []
    for root, _, files in os.walk(logdir):
        #print(root)
        if 'progress.txt' in files:  ## progress txt not use with toy envs, all data in env_params.pkl
            run_name = root[13:]
            exp_name = None
            try:
                config_path = open(os.path.join(root,'config.json'))
                config = json.load(config_path)
                if 'exp_name' in config:
                    exp_name = config['exp_name']
                    
            except:
                print('No file named config.json')
            condition1 = condition or exp_name or 'exp'
            condition2 = condition1 + '-' + str(exp_idx)
            exp_idx += 1
            if condition1 not in units:
                units[condition1] = 0
            unit = units[condition1]
            units[condition1] += 1
            
            performance = 'AverageTestEpRet' if 'AverageTestEpRet' in exp_data else 'AverageEpRet'
            exp_data.insert(len(exp_data.columns),'Unit',unit)
            exp_data.insert(len(exp_data.columns),'Condition1',condition1)
            exp_data.insert(len(exp_data.columns),'Condition2',condition2)
            exp_data.insert(len(exp_data.columns),'evaluation return',exp_data[performance])
            data_dict = exp_data.to_dict("list")
            data_dict['total timesteps'] = []
            for e in data_dict['Epoch']:
                data_dict['total timesteps'].append(e * config['steps_per_epoch'])
            data_dict['config'] = config
            #print(config)
                
            nb_epochs = len(data_dict['total timesteps'])
            print('{} -> {}'.format(run_name, nb_epochs))
            if nb_epochs >= minlen:
                if 'env_params_save.pkl' in files:
                    try:
                        env_params_dict = pickle.load( open(os.path.join(root,'env_params_save.pkl'), "rb" ) )
                    except EOFError:
                        print('Corrupted save, ignoring {}'.format(data_dict['config']['seed']))
                        continue
                    for k,v in env_params_dict.items():
                        #print(k)
#                         if k=="env_test_rewards" or k=='env_params_test' or k=='cegt_means' or k=='cegt_covariances'\
#                            or k=='cegt_episodes' or k=='weights' or k=='cegt_tasks_origin' or k=='cegt_expert_idx'\
#                            or k=='cegt_nb_alpgmm_gaussians' or k=='cegt_initial_expert_means'\
#                            or k=='env_train_len' or k=='cegt_episodes' or k=='selected_expert'\
#                            or k=='cegt_test_vectors' or k=='cegt_k' or k=='cegt_pt' or k=='episodes'\
#                            or k=='env_train_rewards':
                        if True :# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#                         if k=="env_test_rewards" or k=='env_params_test' or k=='episodes'\
#                            or k=="evaluation return" or k=="total timesteps":
                               data_dict[k] = v
                        else:
                            data_dict[k] = 4242
                #print(data_dict.keys())
                datasets.append(data_dict)
    return datasets
#ICLR_data/ignore default final no adapt/
def get_datasets(rootdir):#NeurIPS_2020/20_03_classroom_campain/"):#teachDRL/data/ICLR_data/big_table_data/
    _, models_list, _ = next(os.walk(rootdir))
    print(models_list)
    for dir_name in models_list.copy():
        if "ignore" in dir_name or "elders" in dir_name:
            models_list.remove(dir_name)
    for expe_name in list(labels.keys()):
        if expe_name not in models_list:
            del labels[expe_name]
    
#     # setting specific colors for each expe
#     for i,m_name in enumerate(models_list):
#         if m_name in specific_colors:
#             colors[m_name] = specific_colors[m_name]
#         else:
#             colors[m_name] = default_colors[i]
            
    # setting per-model type colors    
    for i,m_name in enumerate(models_list):
        for m_type, m_color in per_model_colors.items():
            if m_type in m_name:
                colors[m_name] = m_color
        print("extracting data for {}...".format(m_name))
        m_id = m_name
        models_saves[m_id] = OrderedDict()
        models_saves[m_id]['data'] = get_all_runs(rootdir+m_name)
        print("done")
        if m_name not in labels:
            labels[m_name] = m_name

    """
    retrieve all experiences located in "data to vizu" folder
    """
default_colors = ["violet","yellow",'green','black',u'#ff7f0e',
                      "cyan", "pink", u'#1f77b4',"grey","r",
                     "darkorchid","sienna","lightpink","blue","magenta", "indigo","mediumseagreen",'aqua',
                'deeppink','silver','khaki','goldenrod','y','y','y','y','y','y','y','y','y','y','y','y' ]  + ['y']*50
print(len(default_colors))
labels = OrderedDict()
per_model_colors = OrderedDict()

# LOAD DATA
models_saves = OrderedDict()
colors = OrderedDict()
get_datasets("teachDRL/data/")
print(per_model_colors)
print(labels)
if per_model_colors:  # order runs for legend order as in per_models_colors, with corresponding colors
    ordered_labels = OrderedDict()
    for teacher_type in per_model_colors.keys():
        for k,v in labels.items():
            if teacher_type in k:
                ordered_labels[k] = v
    labels = ordered_labels
else:
    print('not using per_model_color')
    for k in models_saves.keys():
        labels[k] = k
        
print(labels)
print(colors)

In [ ]:
################################## BUILD CLASSROOM #############################################

# Construct history of trained students
def load_expert_trajectory(data, alp_thr=0.2):
    # 1 - loading the trajectory
    for k, v in data.items():
        if k == 'means' or k == 'covariances' or k == 'weights' or k == 'env_train_len' or k == 'episodes' \
           or k == 'env_train_rewards':
            data[k] = v
        if k == 'egt_means' or k == 'egt_covariances' or k == 'egt_weights' or k == 'egt_env_train_len' or k == 'egt_episodes' \
                or k == 'egt_env_train_rewards':
            data[k[4:]] = v

    # 2 - pre-processing expert trajectory
    # removing low-alp gaussians
    processed_gmms_means = []
    processed_gmms_covs = []
    processed_gmms_mean_rew = []
    idx_removed_gmms = []
    #print(data['episodes'])
    gmm_step = data['episodes'][0]
    max_lp = 0.0
    for i, (gmm_means, gmm_covs, episode) in enumerate(zip(data["means"], data["covariances"], data['episodes'])):
        #step_nb += sum(data['env_train_len'][i*gmm_step:(i+1)*gmm_step])
        processed_gmm_means = []
        processed_gmm_covs = []
        all_rewards = data['env_train_rewards'][episode:episode + gmm_step] # from gmm
        rewards = all_rewards[-50:]  # consider mean reward after some training on the GMM
        mean_reward = np.mean(rewards)
        for j, (means, covs) in enumerate(zip(gmm_means, gmm_covs)):
            if means[-1] > max_lp:
                max_lp = means[-1]
            if means[-1] > alp_thr:  # last mean is ALP dimension
                # add gaussian
                processed_gmm_means.append(means)
                processed_gmm_covs.append(covs)
        if not processed_gmm_means == []:  # gmm not empty after pre-process, lets add it
            processed_gmms_means.append(processed_gmm_means)
            processed_gmms_covs.append(processed_gmm_covs)
            processed_gmms_mean_rew.append(mean_reward)
        else:
            idx_removed_gmms.append(i)
#     print('idx of removed gmms ({}/{}) in expert traj: {}, max_lp={}'.format(len(data['means']) - len(processed_gmms_means),
#                                                                   len(data['means']),
#                                                                   idx_removed_gmms, max_lp))
    #print('number of steps: {}'.format(step_nb))
    return len(processed_gmms_means)



folder = 'teachDRL/graphics/elders/raw/'

import matplotlib.colorbar as cbar
import teachDRL.teachers.utils.plot_utils as plotter
import pickle

ignore_failed_trainings = True  # SET TO TRUE WHEN BUILDING CLASSROOM
keep_no_progress = True
student_ids = []
#student_params = {'leg_s':[], 'is_quad':[]}
student_params = {'agent_type':[], 'leg_s':[], 'nn':[]}
initial_test_vectors_list = [[],[],[],[],[]] # lists of first test vectors from idx 0 to 4 (ie. 0.5M to 2.5M)
last_test_vector = []
last_perfs = []
max_lps = []



classroom_m_ids = ['PUT CLASSROOM NAME HERE']
                
nb_no_progress = 0
print(labels.keys())
for i,(m_id,label) in enumerate(labels.items()):
    if not(m_id in classroom_m_ids):
        continue
    print(m_id)
    runs_data = models_saves[m_id]['data']
    all_alpgmm_tasks_percentage = []
    nb_zero_perf = 0
#    nb_zero_perf_small_net = 0
#    nb_small_net = 0
    for r,run in enumerate(runs_data):
        assert(len(run['evaluation return']) == 20)
        test_env_vectors = []
        last_perf = round(run["custom_nb_mastered"][-1],1) if 'custom_nb_mastered' in run.keys() else round(run["nb_mastered"][-1],1)
        max_perf = round(max(run["nb_mastered"]),1)
        perf_str = "last:{}, max:{}".format(last_perf, max_perf)
        if last_perf == 0.0:
            nb_zero_perf += 1
            max_lps.append(load_expert_trajectory(run, alp_thr=0.2))
        perf_title = "Last perf: {}% | Max perf: {}%".format(last_perf, max_perf)

        tests_rewards = np.array_split(run['env_test_rewards'],20)
        initial_score = np.round(sum([np.interp(r, (-150, 350), (0, 1)) for r in tests_rewards[0]]),2)
        last_score = np.round(sum([np.interp(r, (-150, 350), (0, 1)) for r in tests_rewards[-1]]),2)

        b_type = 0 if 'biped' in run['config']['env_init']['agent_body_type'] else 1
        leg_s = run['config']['env_init']['leg_s']
        nn = run['config']['ac_kwargs']['hidden_sizes'][0]
        print('seed: {}, leg_s:{},b_type:{}, last_perf: {} score={}-->{}: imp:{}'.format(run['config']['seed'],
                                                               leg_s, b_type,
                                                                   last_perf,
                                                                   initial_score,last_score,
                                                                    last_score >= initial_score))
        
        test_params = np.array_split(run['env_params_test'],20)[0]
        len_expert_traj = load_expert_trajectory(run, alp_thr=0.2)
        if ((last_score >= initial_score or keep_no_progress) and len_expert_traj > 0) or not ignore_failed_trainings: # use in history only if the student managed to progress and has high-lp gaussians
            last_perfs.append(last_perf)
            student_ids.append(m_id + '_s' + str(run['config']['seed']))
            for idx,rews in enumerate(tests_rewards):
                test_env_vectors.append(np.array([np.interp(r, (-150, 350), (0, 1)) for r in rews]))
                if idx <= 4:
                    initial_test_vectors_list[idx].append(np.array([np.interp(r, (-150, 350), (0, 1)) for r in rews]))
                if idx == len(tests_rewards)-1:
                    last_test_vector.append(np.array([np.interp(r, (-150, 350), (0, 1)) for r in rews]))
            #student_params['leg_s'].append(leg_s)
            student_params['agent_type'].append(b_type)
            student_params['leg_s'].append(leg_s)
            student_params['nn'].append(nn)
        else:
            nb_no_progress += 1
            if last_score <= initial_score and len_expert_traj == 0:
                print('do not use {}: no progress and no high-lp gaussians'.format(m_id))
            elif last_score <= initial_score:
                print('do not use {}: no progress'.format(m_id))
            elif len_expert_traj == 0:
                print('do not use {}: no high-lp gaussians'.format(m_id))

print('{}/{} failed learning, removed from classroom'.format(nb_no_progress,len(runs_data)))
print('{} in classroom'.format(len(initial_test_vectors_list[0])))
if ignore_failed_trainings:
    pickle.dump([student_ids, initial_test_vectors_list, last_test_vector, last_perfs, student_params],
                open( "teachDRL/data/elders_knowledge/parkour_classroom.pkl", "wb" ))
        